In [23]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb


from path_setup import setup_paths
setup_paths()

In [24]:
from dataset_transformer import BaseDatasetTransform
from base_model_train import BaseModelTrain
from model_evaluator import ModelEvaluator

In [25]:
df = pd.read_csv('../datasets/ServreqPitt.csv')

In [26]:
df.head()

,REQUEST_ID,CREATED_ON,REQUEST_TYPE,REQUEST_ORIGIN,STATUS,DEPARTMENT,NEIGHBORHOOD,COUNCIL_DISTRICT,WARD,TRACT,PUBLIC_WORKS_DIVISION,PLI_DIVISION,POLICE_ZONE,FIRE_ZONE,X,Y,GEO_ACCURACY
0,168714,2017-07-10T14:32:00,Fire Safety System Not Working,Control Panel,1,"Permits, Licenses and Inspections",Carrick,4.0,29.0,4.200329e+10,3.0,29.0,3.0,4-15,-79.980279,40.388412,APPROXIMATE
1,251339,2018-05-22T12:12:00,Illegal Dumping,Website,1,DOMI - Permits,Garfield,9.0,10.0,4.200310e+10,2.0,10.0,5.0,3-4,-79.938336,40.466358,APPROXIMATE
2,168709,2017-07-10T14:29:00,Potholes,Call Center,1,DPW - Street Maintenance,Polish Hill,7.0,6.0,4.200306e+10,6.0,6.0,2.0,2-6,-79.965908,40.457074,EXACT
3,168710,2017-07-10T14:30:00,Abandoned Vehicle (parked on street),Call Center,0,Police - AVU,Marshall-Shadeland,1.0,27.0,4.200327e+10,1.0,27.0,1.0,1-14,-80.033916,40.471775,APPROXIMATE
4,189977,2017-09-27T17:02:00,Manhole Cover,Call Center,0,DOMI - Permits,Central Business District,6.0,2.0,4.200302e+10,6.0,2.0,2.0,1-3,-80.000264,40.443240,EXACT


In [27]:
df['CREATED_ON'] = pd.to_datetime(df['CREATED_ON'])



In [28]:
df['hour_sin'] = np.sin(2 * np.pi * df['CREATED_ON'].dt.hour / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['CREATED_ON'].dt.hour / 24)

df['weekday_sin'] = np.sin(2 * np.pi * df['CREATED_ON'].dt.weekday / 7)
df['weekday_cos'] = np.cos(2 * np.pi * df['CREATED_ON'].dt.weekday / 7)

df['month'] = df['CREATED_ON'].dt.month

df['day'] = df['CREATED_ON'].dt.day

df['year'] = df['CREATED_ON'].dt.year

df = df.drop('CREATED_ON',axis=1)

In [29]:
bdf = BaseDatasetTransform(df,target='REQUEST_TYPE')

In [30]:
df,categorical_features = bdf.fit_transform()


Обнаружены колонки с именем "id": ['REQUEST_ID']
Колонка "REQUEST_ID" удалена.
Series([], dtype: int64)

В колонке(ах) ['REQUEST_TYPE'] нет пропущенных значений
-------------------------------------------
В наборе данных есть пропущенные значения:
Колонки с пропущенными значениями:
DEPARTMENT                2201
NEIGHBORHOOD             22157
COUNCIL_DISTRICT         21772
WARD                     22077
TRACT                    20952
PUBLIC_WORKS_DIVISION    22070
PLI_DIVISION             22077
POLICE_ZONE              22245
FIRE_ZONE                21924
X                        18628
Y                        18628
dtype: int64
Все пропущенные значения заполнены.
-------------------------------------------
Информация о колонках в датасете

Категориальные колонки:
Index(['REQUEST_ORIGIN', 'DEPARTMENT', 'NEIGHBORHOOD', 'FIRE_ZONE',
       'GEO_ACCURACY'],
      dtype='object')

Числовые колонки:
Index(['STATUS', 'COUNCIL_DISTRICT', 'WARD', 'TRACT', 'PUBLIC_WORKS_DIVISION',
       'PLI_

In [32]:
X_train, X_test, y_train, y_test  = bdf.get_train_test_split()

Количество значений целевой переменной по категориям:
REQUEST_TYPE
Potholes                            41169
Weeds/Debris                        25309
Building Maintenance                13815
Snow/Ice removal                     9063
Refuse Violations                    8271
                                    ...  
Ethics Office                           2
Benefits                                1
Handicapped Parking Sign Request        1
Amusement Tax                           1
Sidewalk (city property)                1
Name: count, Length: 294, dtype: int64
Следует ли выполнить стратифицированное раздеение на обучающую и тестовую выборку? y/n
Разделение датасета выполнено успешно


In [33]:
lgbm_model = lgb.LGBMClassifier(verbose=-1)

columns_to_delete = []  

unique_classes = len(set(y_train))
print(unique_classes)

model_train = BaseModelTrain(
    model=lgbm_model,
    columns_to_delete=columns_to_delete,
    categorical_features=categorical_features,
    boosting_type='gbdt',
    objective='multiclass',  
    num_class=unique_classes, 
    metric='multi_error' 
)

293


In [34]:
metrics = ['accuracy', 'precision', 'recall', 'f1_macro']
evaluator = ModelEvaluator(model_trainer=model_train, metrics=metrics)

evaluator.fit(X_train, y_train)

ModelEvaluator(metrics=['accuracy', 'precision', 'recall', 'f1_macro'],
               model_trainer=BaseModelTrain(categorical_features=['REQUEST_ORIGIN',
                                                                  'DEPARTMENT',
                                                                  'NEIGHBORHOOD',
                                                                  'FIRE_ZONE',
                                                                  'GEO_ACCURACY'],
                                            columns_to_delete=[],
                                            model=LGBMClassifier(metric='multi_error',
                                                                 num_class=293,
                                                                 objective='multiclass',
                                                                 verbose=-1)))

In [37]:
y_test

31838                     Bulk
155473                Potholes
95151     Building Maintenance
37715                  Traffic
270812         Vacant Building
                  ...         
91103                 Potholes
170930    Building Maintenance
248606       Abandoned Vehicle
78162     Building Maintenance
195347      Bridge Maintenance
Name: REQUEST_TYPE, Length: 86187, dtype: object

In [35]:
evaluator.evaluate_to_dataframe(X_test, y_test)

AttributeError: 'Series' object has no attribute 'columns'